# Extracting years and coordinates from json files

In [1]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime
from matplotlib import pyplot as plt

In [ ]:
#s3_resource = boto3.resource('s3')
## Copy object A as object B
#for 
#s3_resource.Object('streetview-w210', 'gsv/').copy_from(
# CopySource=”path/to/your/object_A.txt”)
## Delete the former object A
#s3_resource.Object(“bucket_name”, “path/to/your/object_A.txt”).delete()

In [4]:
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3')
fnames = fs.ls('s3://streetview-w210/gsv/')
fnames = [x for x in fnames if x.endswith('.json')]
fnames2 = fs.ls('s3://streetview-w210/')
fnames2 = [x for x in fnames2 if x.endswith('.json')]
fnames = fnames + fnames2
len(fnames)

19615

In [5]:
bucket_name = 'streetview-w210'
dates, lat, long = [],[],[]

start = datetime.now()

for i,fname in enumerate(fnames):
    json_path = 's3://{}'.format(fname)
    obj = s3.get_object(Bucket=bucket_name, Key=fname[len(bucket_name)+1:])
    json_file = obj['Body'].read().decode('utf-8')
    
    file_data = json.loads(json_file)
    if 'date' not in file_data:
        continue
    dates.append(file_data['date'])
    lat.append(file_data['location']['lat'])
    long.append(file_data['location']['lng'])
    
print(datetime.now()-start)

df = pd.DataFrame([dates,lat,long]).transpose()
df.columns = ['date','lat','long']

0:11:10.078211


In [6]:
df.to_csv('years.csv', index=False)